<a href="https://colab.research.google.com/github/davidislip/SVM-and-MVO/blob/main/ADM_Comparison_NoteBook_Timing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Integration of SVM and MVO 

This notebook performs a comparison of the integrated MVO-SVM min variance portfolios obtained via the exact MIP solve vs the ADM method


In [ ]:
!sudo python -m pip install gurobipy==9.1.2
import gurobipy as gp
from gurobipy import GRB
from google.colab import drive 
import math
import time
drive.mount('/content/gdrive')
pth = 'gdrive/My Drive/Colab Notebooks/'
import sys
sys.path.append(pth + 'SVM MVO/PYFILES/')
from packages import *
#from svm_mvo import *
%matplotlib inline

## Import Forecasts
forecasts = pd.read_pickle(pth +'SVM MVO/DATA/Forecasts.pkl')
## Import Returns
rets = pd.read_pickle(pth + 'SVM MVO/DATA/RETS.pkl')
## Import feature vectors
wrds_svm = pd.read_pickle(pth + 'SVM MVO/DATA/WRDS_ASOF.pkl')
wrds_svm_pca = pd.read_pickle(pth + 'SVM MVO/DATA/WRDS_PCA.pkl')
wrds_svm_pca['Momentum'] = wrds_svm.Momentum
## Import Monthly Prices
prices = pd.read_pickle(pth + 'SVM MVO/DATA/PRICES.pkl')

     |████████████████████████████████| 11.1 MB 6.7 MB/s 
Mounted at /content/gdrive


In [ ]:
try:
  e = gp.Env(empty=True)
  #e.setParam('OutputFlag', 0)
  gurobi_licence = pd.read_csv(pth +'SVM MVO/DATA/gurobi.csv')
  e.setParam('WLSACCESSID', gurobi_licence.WLSACCESSID[0])
  e.setParam('LICENSEID', gurobi_licence.LICENSEID[0])
  e.setParam('WLSSECRET', gurobi_licence.WLSSECRET[0])
  e.start()
except: 
  !chmod 755 /content/gdrive/My\ Drive/Colab\ Notebooks/SVM\ MVO/gurobi/grbgetkey
  !/content/gdrive/My\ Drive/Colab\ Notebooks/SVM\ MVO/gurobi/grbgetkey 2d15bcec-464e-11ec-8a74-0242ac120003
  e = gp.Env(empty=True)
  #chmod 755 grbgetkey
  e.start()


Changed value of parameter WLSACCESSID
Changed value of parameter LICENSEID
Changed value of parameter WLSSECRET
Academic license - for non-commercial use only - registered to ryan.islip@mail.utoronto.ca
Using license file /usr/local/lib/python3.7/dist-packages/gurobipy/.libs/gurobi.lic
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID


In [ ]:
  !df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         226G   42G  184G  19% /
tmpfs            64M     0   64M   0% /dev
tmpfs           6.4G     0  6.4G   0% /sys/fs/cgroup
shm             5.8G     0  5.8G   0% /dev/shm
/dev/root       2.0G  1.2G  821M  59% /sbin/docker-init
tmpfs           6.4G   36K  6.4G   1% /var/colab
/dev/sda1       233G   45G  188G  20% /etc/hosts
tmpfs           6.4G     0  6.4G   0% /proc/acpi
tmpfs           6.4G     0  6.4G   0% /proc/scsi
tmpfs           6.4G     0  6.4G   0% /sys/firmware
drive            15G  5.1G   10G  34% /content/gdrive


In [ ]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 b

In [ ]:
!cat /proc/meminfo

MemTotal:       13302924 kB
MemFree:         9334272 kB
MemAvailable:   12059500 kB
Buffers:          145468 kB
Cached:          2665364 kB
SwapCached:            0 kB
Active:          1408400 kB
Inactive:        2314320 kB
Active(anon):     823684 kB
Inactive(anon):      452 kB
Active(file):     584716 kB
Inactive(file):  2313868 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:              7180 kB
Writeback:             0 kB
AnonPages:        912204 kB
Mapped:           290484 kB
Shmem:              1152 kB
KReclaimable:     136240 kB
Slab:             180808 kB
SReclaimable:     136240 kB
SUnreclaim:        44568 kB
KernelStack:        5872 kB
PageTables:         8184 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6651460 kB
Committed_AS:    4118276 kB
VmallocTotal:   34359738367 kB
VmallocUsed:        8376 kB
VmallocChunk:          0 kB
Percpu:          

In [ ]:
%run /content/gdrive/My\ Drive/Colab\ Notebooks/SVM\ MVO/PYFILES/svm_mvo.py

Changed value of parameter WLSACCESSID
Changed value of parameter LICENSEID
Changed value of parameter WLSSECRET
Academic license - for non-commercial use only - registered to ryan.islip@mail.utoronto.ca
Using license file /usr/local/lib/python3.7/dist-packages/gurobipy/.libs/gurobi.lic
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID


In [ ]:
mth = rets.index[0]
mth

Timestamp('2000-07-01 00:00:00')

In [ ]:
#convenient function
sign = lambda a: int((a>0)) - int((a<0))

#preprocessing 
cov_, mean_ = forecasts[mth]
N = 200 #len(mean_)
cov = cov_[:N,:N]
mean = mean_[:N,:]
tics = list(rets.columns[:N])
print("valid tickers:", tics)
#get the wharton research data for the valid tickers for the month
wrds_tics = wrds_svm[wrds_svm.index.get_level_values('tic').isin(tics)].xs(mth, level="MonthStart")
#restrict the wharton research data to the columns of interest
Y = wrds_tics.loc[:,"ROC1":"Momentum"] # Y matrix in formulation 
Y_ =  (Y - Y.mean(axis=0))/(Y.std(axis=0))
Y_ = Y_.iloc[:,:]
#Y_ = Y_[['ROC1', 'INV1', 'PRO1'	, 'GRW1', 'LEV1', 'LIQ1']]
#Y_['momentum'] = mean
cols = ['LEV1', 'GRW1', 'Momentum']
Y_ = Y_[cols]
#Y_['momentum'] = mean
AssetLim = math.floor(N*0.8)

ret_constr = -1
soft_margin = 2000

valid tickers: ['AAPL', 'ABC', 'ABMD', 'ABT', 'ADBE', 'ADI', 'ADM', 'ADP', 'ADSK', 'AEE', 'AEP', 'AES', 'AJG', 'AKAM', 'ALB', 'ALK', 'AMAT', 'AMD', 'AME', 'AMGN', 'AMT', 'AMZN', 'ANSS', 'AON', 'AOS', 'APA', 'APD', 'APH', 'ATO', 'AVY', 'AZO', 'BA', 'BAX', 'BBY', 'BDX', 'BKNG', 'BLL', 'BMY', 'BSX', 'BWA', 'CAG', 'CAH', 'CAT', 'CCI', 'CCL', 'CDNS', 'CERN', 'CHD', 'CHRW', 'CI', 'CL', 'CLX', 'CMCSA', 'CMI', 'CMS', 'CNP', 'COO', 'COP', 'COST', 'CPB', 'CPRT', 'CSCO', 'CSX', 'CTAS', 'CTSH', 'CTXS', 'CVS', 'CVX', 'D', 'DD', 'DGX', 'DHR', 'DIS', 'DISH', 'DLTR', 'DOV', 'DRI', 'DTE', 'DUK', 'DVA', 'DVN', 'DXC', 'EA', 'EBAY', 'ECL', 'ED', 'EFX', 'EIX', 'EL', 'EMN', 'EMR', 'EOG', 'ES', 'ETN', 'ETR', 'EVRG', 'EXC', 'EXPD', 'F', 'FAST', 'FCX', 'FDX', 'FE', 'FFIV', 'FISV', 'FLS', 'FMC', 'GD', 'GIS', 'GLW', 'GPC', 'GPS', 'GWW', 'HAL', 'HAS', 'HD', 'HES', 'HFC', 'HOG', 'HOLX', 'HON', 'HP', 'HPQ', 'HRB', 'HRL', 'HSIC', 'HSY', 'HUM', 'IBM', 'IDXX', 'IEX', 'IFF', 'INCY', 'INTC', 'INTU', 'IP', 'IPG', 'IRM', 

### Mean Variance and SVM Joint Using Alternating Directions Method Class Deinition

In [ ]:
MVO_ = MVO(tics, mean, cov, ret_constr, Y_, AssetLim)
SVM_ = SVM(tics, Y_ , soft_margin)
SVM_MVO_Fast = SVM_MVO_ADM(MVO_, SVM_, IterLim = 200)
SVM_MVO_Fast.MVO_.model.params.OutputFlag = 0
SVM_MVO_Fast.SVM_.model.params.OutputFlag = 0
start = time.time()
SVM_MVO_Fast.initialize_soln()
ws , xs, zs , xi_mvo, xi_svm, dt = SVM_MVO_Fast.solve_ADM()
end = time.time()
print("Solution time", end - start)


Academic license - for non-commercial use only - registered to ryan.islip@mail.utoronto.ca
Academic license - for non-commercial use only - registered to ryan.islip@mail.utoronto.ca
Academic license - for non-commercial use only - registered to ryan.islip@mail.utoronto.ca
Academic license - for non-commercial use only - registered to ryan.islip@mail.utoronto.ca
Academic license - for non-commercial use only - registered to ryan.islip@mail.utoronto.ca
Academic license - for non-commercial use only - registered to ryan.islip@mail.utoronto.ca
Solution time 2.7023961544036865


##Exact MIP Solution

In [ ]:
#SVM with slack 
SVM_MVO_Slck = SVMMVO(tics, mean, cov, ret_constr, soft_margin, Y_, AssetLim, \
                      svm_choice = (True, True), print_var_frntr = False, indicator = False)
SVM_MVO_Slck.setModel()
# SVM_MVO_Slck.model.params.Presolve = 2
# SVM_MVO_Slck.model.Params.ImpliedCuts = 2
SVM_MVO_Slck.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Academic license - for non-commercial use only - registered to ryan.islip@mail.utoronto.ca
Optimize a model with 803 rows, 804 columns and 3800 nonzeros
Model fingerprint: 0x4de917d2
Model has 20103 quadratic objective terms
Variable types: 604 continuous, 200 integer (200 binary)
Coefficient statistics:
  Matrix range     [3e-04, 1e+05]
  Objective range  [1e+01, 1e+01]
  QObjective range [1e-06, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-03, 2e+02]
Found heuristic solution: objective 4.000000e+12
Presolve removed 1 rows and 200 columns
Presolve time: 0.01s
Presolved: 802 rows, 604 columns, 3600 nonzeros
Presolved model has 20103 quadratic objective terms
Variable types: 404 continuous, 200 integer (200 binary)

Root relaxation: objective 7.777133e-04, 490 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bo

116.9431791305542

## Comparison between exact and ADM method

In [ ]:
return_premium = -1 #(Min variance portfolio)
T = len(rets.index)

In [ ]:
import time
def cb(model, where):
    if where == GRB.Callback.MIPNODE:
        # Get model objective
        obj = model.cbGet(GRB.Callback.MIPNODE_OBJBST)

        # Has objective changed?
        if abs(obj - model._cur_obj) > 1e-8:
            # If so, update incumbent and time
            model._cur_obj = obj
            model._time = time.time()

    # Terminate if objective has not improved in 60s
    if time.time() - model._time > 60:
        model.terminate()

def evaluate_model(rets, forecasts, wrds_svm, return_premium, model_instance, T, N, cols):
  # lr = 0.1 #learning rate for the SVM support vector
  portfolio_weights = np.zeros([T,N])
  oot_returns = np.zeros(T)
  market = np.zeros(T)
  M = len(cols)
  wis = np.zeros([T,M])
  bias = np.zeros(T)
  times = []
  i = 0 #index for dates
  w_mabs = 0 #initialize
  # soln_mods = {}

  for prd in rets.index.to_list()[:T]:
    ret_ = rets.loc[prd][:N]
    cov_, mean_ = forecasts[prd]
    cov = cov_[:N,:N]
    mean = mean_[:N,:]
    tics = list(rets.columns[:N])
    return_premium_temp = return_premium 
    #get the wharton research data for the valid tickers for the month
    wrds_tics = wrds_svm[wrds_svm.index.get_level_values('tic').isin(tics)].xs(prd, level="MonthStart")
    #restrict the wharton research data to the columns of interest
    #modifying WRDS dataset here if required
    Y = wrds_tics.loc[:,cols] # Y matrix in formulation 
    Y_ =  (Y - Y.mean(axis=0))/(Y.std(axis=0))
    if return_premium == -1:
      ret_constr = -1
    else:
      ret_constr = mean.mean()*(1 + sign(mean.mean())*return_premium)
    model_instance.tics = tics
    model_instance.ret_constr = ret_constr
    model_instance.mean_ret = mean
    model_instance.cov = cov
    model_instance.exogenous = Y_

    # if i > 0: #not the first trade gets a constraint on turnover
    #   x_prev = model_instance.x.X

    # if model_instance.svm_constr:
    #   w_mabs = (i/(i+1))*w_mabs + (1/(i+1))*np.abs(model_instance.w.x).mean()
    #   w_prev = model_instance.w.x

    model_instance.setModel()

    # if i > 0: #not the first trade gets a constraint on turnover
    #   model_instance.define_turnover(x_prev, np.ones_like(x_prev), Limit, 1)

    # if model_instance.svm_constr:
    #   u = model_instance.model.addMVar(M, lb = np.zeros(M), ub = np.ones(M))
    #   wcon1 = model_instance.model.addConstr(model_instance.w <= w_prev + lr*w_mabs*u + model_instance.epsilon, 'iter constraint 1')
    #   wcon2 = model_instance.model.addConstr(model_instance.w >= w_prev - lr*w_mabs*u - model_instance.epsilon, 'iter constraint 2')

    model_instance.model.Params.LogToConsole = 0
    model_instance.model.setParam('TimeLimit', 200)
    start = time.time()
    model_instance.optimize()
    end = time.time()
    dt = end - start 
    print(dt)
    # k = 1
    # while model_instance.model.status == 4:
    #   #if the model is infeasible the decrease the return constraint
    #   #we do not have enough turnover the modify the portfolio to achive the 
    #   #return target... not a great place to be
    #   return_premium_temp = return_premium - 0.05*k
    #   ret_constr = mean.mean()*(1 + sign(mean.mean())*return_premium_temp)
    #   model_instance.ret_constr = ret_constr
    #   model_instance.ret_target[0].rhs = ret_constr
    #   model_instance.optimize(cbb = cb)
    #   k = k+1
      #relax the return target bc the turnover constraint isnt allowing it to solve


    # if model_instance.model.IsMIP:
    #   soln_mods[prd] = [model_instance.model.MIPGap, return_premium_temp]
    # else:
    #   soln_mods[prd] = return_premium_temp
    
    oot_returns[i] = model_instance.evaluate(ret_)
    market[i] = ret_.mean()
    portfolio_weights[i, :] = model_instance.x.x
    times.append(dt)

    if model_instance.svm_constr:
      wis[i,:] = model_instance.w.x
      bias[i] = model_instance.b.x
    if i + 1 >= T:
      break
    if i%12 == 0:
      print("_"*25)
      print("Iteration ", i)
      print("Percent Complete ", i/T)
    i = i+1
  
  portfolio_weights = pd.DataFrame(portfolio_weights, index = rets.index[:T], columns = model_instance.tics)
  oot_returns = pd.Series(oot_returns, index = rets.index[:T])
  market = pd.Series(market, index = rets.index[:T])
  return (portfolio_weights, oot_returns, market, wis, bias, times)


In [ ]:
svm_results = evaluate_model(rets, forecasts, wrds_svm, return_premium, SVM_MVO_Slck, T, N, cols)
with open("gdrive/My Drive/Colab Notebooks/SVM MVO/DATA/svm_exact_mvp_lrg.pkl", 'wb') as fp:
  pkl.dump(svm_results, fp);


Academic license - for non-commercial use only - registered to ryan.islip@mail.utoronto.ca
200.0572018623352
_________________________
Iteration  0
Percent Complete  0.0
Academic license - for non-commercial use only - registered to ryan.islip@mail.utoronto.ca
88.5223650932312
Academic license - for non-commercial use only - registered to ryan.islip@mail.utoronto.ca
145.3525505065918
Academic license - for non-commercial use only - registered to ryan.islip@mail.utoronto.ca
47.737929344177246
Academic license - for non-commercial use only - registered to ryan.islip@mail.utoronto.ca
19.28276038169861
Academic license - for non-commercial use only - registered to ryan.islip@mail.utoronto.ca
13.90980839729309
Academic license - for non-commercial use only - registered to ryan.islip@mail.utoronto.ca
36.72337031364441
Academic license - for non-commercial use only - registered to ryan.islip@mail.utoronto.ca
68.83245062828064
Academic license - for non-commercial use only - registered to ryan

In [ ]:

def evaluate_adm(rets, forecasts, wrds_svm, return_premium, model_instance, T, N, cols):
  # lr = 0.1 #learning rate for the SVM support vector
  portfolio_weights = np.zeros([T,N])
  oot_returns = np.zeros(T)
  market = np.zeros(T)
  M = len(cols)
  wis = np.zeros([T,M])
  bias = np.zeros(T)
  times = []
  i = 0 #index for dates
  w_mabs = 0 #initialize
  # soln_mods = {}

  for prd in rets.index.to_list()[:T]:
    ret_ = rets.loc[prd][:N]
    cov_, mean_ = forecasts[prd]
    cov = cov_[:N,:N]
    mean = mean_[:N,:]
    tics = list(rets.columns[:N])
    return_premium_temp = return_premium 
    #get the wharton research data for the valid tickers for the month
    wrds_tics = wrds_svm[wrds_svm.index.get_level_values('tic').isin(tics)].xs(prd, level="MonthStart")
    #restrict the wharton research data to the columns of interest
    #modifying WRDS dataset here if required
    Y = wrds_tics.loc[:,cols] # Y matrix in formulation 
    Y_ =  (Y - Y.mean(axis=0))/(Y.std(axis=0))
    if return_premium == -1:
      ret_constr = -1
    else:
      ret_constr = mean.mean()*(1 + sign(mean.mean())*return_premium)
    model_instance.MVO_.tics = tics
    model_instance.SVM_.tics = tics
    model_instance.MVO_.ret_constr = ret_constr
    model_instance.MVO_.mean_ret = mean
    model_instance.MVO_.cov = cov
    model_instance.MVO_.exogenous = Y_
    model_instance.SVM_.exogenous = Y_
    # if i > 0: #not the first trade gets a constraint on turnover
    #   x_prev = model_instance.x.X

    # if model_instance.svm_constr:
    #   w_mabs = (i/(i+1))*w_mabs + (1/(i+1))*np.abs(model_instance.w.x).mean()
    #   w_prev = model_instance.w.x
    # if i == 0:
    start = time.time()
    model_instance.initialize_soln()
    # else:
    #   start = time.time()

    # if i > 0: #not the first trade gets a constraint on turnover
    #   model_instance.define_turnover(x_prev, np.ones_like(x_prev), Limit, 1)

    # if model_instance.svm_constr:
    #   u = model_instance.model.addMVar(M, lb = np.zeros(M), ub = np.ones(M))
    #   wcon1 = model_instance.model.addConstr(model_instance.w <= w_prev + lr*w_mabs*u + model_instance.epsilon, 'iter constraint 1')
    #   wcon2 = model_instance.model.addConstr(model_instance.w >= w_prev - lr*w_mabs*u - model_instance.epsilon, 'iter constraint 2')

    model_instance.silence_output()
    model_instance.solve_ADM(store_data=False)
    end = time.time()
    dt = end-start
    print(dt)
    # k = 1
    # while model_instance.model.status == 4:
    #   #if the model is infeasible the decrease the return constraint
    #   #we do not have enough turnover the modify the portfolio to achive the 
    #   #return target... not a great place to be
    #   return_premium_temp = return_premium - 0.05*k
    #   ret_constr = mean.mean()*(1 + sign(mean.mean())*return_premium_temp)
    #   model_instance.ret_constr = ret_constr
    #   model_instance.ret_target[0].rhs = ret_constr
    #   model_instance.optimize(cbb = cb)
    #   k = k+1
      #relax the return target bc the turnover constraint isnt allowing it to solve

    # if model_instance.model.IsMIP:
    #   soln_mods[prd] = [model_instance.model.MIPGap, return_premium_temp]
    # else:
    #   soln_mods[prd] = return_premium_temp
    
    oot_returns[i] = model_instance.evaluate(ret_)
    market[i] = ret_.mean()
    portfolio_weights[i, :] = model_instance.x.x
    times.append(dt)
    wis[i,:] = model_instance.w.x
    bias[i] = model_instance.b.x

    if i + 1 >= T:
      break
      
    if i%12 == 0:
      print("_"*25)
      print("Iteration ", i)
      print("Percent Complete ", i/T)
    i = i+1
  
  portfolio_weights = pd.DataFrame(portfolio_weights, index = rets.index[:T], columns = model_instance.tics)
  oot_returns = pd.Series(oot_returns, index = rets.index[:T])
  market = pd.Series(market, index = rets.index[:T])
  return (portfolio_weights, oot_returns, market, wis, bias, times)

In [ ]:
adm_results = evaluate_adm(rets, forecasts, wrds_svm, return_premium, SVM_MVO_Fast, T, N, cols)
with open("gdrive/My Drive/Colab Notebooks/SVM MVO/DATA/svm_adm_mvp_lrg.pkl", 'wb') as fp:
  pkl.dump(adm_results, fp);

Academic license - for non-commercial use only - registered to ryan.islip@mail.utoronto.ca
Academic license - for non-commercial use only - registered to ryan.islip@mail.utoronto.ca
Academic license - for non-commercial use only - registered to ryan.islip@mail.utoronto.ca
Academic license - for non-commercial use only - registered to ryan.islip@mail.utoronto.ca
Academic license - for non-commercial use only - registered to ryan.islip@mail.utoronto.ca
Academic license - for non-commercial use only - registered to ryan.islip@mail.utoronto.ca
3.7678308486938477
_________________________
Iteration  0
Percent Complete  0.0
Academic license - for non-commercial use only - registered to ryan.islip@mail.utoronto.ca
Academic license - for non-commercial use only - registered to ryan.islip@mail.utoronto.ca
Academic license - for non-commercial use only - registered to ryan.islip@mail.utoronto.ca
Academic license - for non-commercial use only - registered to ryan.islip@mail.utoronto.ca
Academic l

In [ ]:
pd.DataFrame(np.array(adm_results[5]) - np.array(svm_results[5]), index = rets.index).hist(bins = 30);

In [ ]:
(100*(adm_results[0] - svm_results[0]).abs().mean(axis=1)).hist(bins = 30)